# Build an Agent

By themselves, language models can't take actions - they just output text.
A big use case for LangChain is creating **agents**.
Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.
After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In this tutorial we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

## 1. Concepts

In following this tutorial, you will learn how to:

- Use [language models](/docs/concepts/#chat-models), in particular their tool calling ability
- Use a Search [Tool](/docs/concepts/#tools) to look up information from the Internet
- Compose a [LangGraph Agent](/docs/concepts/#agents), which use an LLM to determine actions and then execute them
- Debug and trace your application using [LangSmith](/docs/concepts/#langsmith)



## 2. End-to-end agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use. It is equipped with a generic search tool. It has conversational memory - meaning that it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does - but if you want to just grab some code and get started, feel free to use this!

In [1]:
import json
from dotenv import load_dotenv
load_dotenv('.env', verbose=True)

# Import relevant functionality
import langgraph
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatOpenAI(model_name="gpt-4o")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

agent_executor


CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 0x7fce391a3ad0>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']), ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)]), 'agent': PregelNode(config={'tags': []}, channels={'messages': 'messages', 'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>}, triggers=['start:agent', 'tools'], mapper=functools.partial(<function _coerce_state at 0x7fce395ba700>, <class 'langgraph.prebuilt.chat_agent_executor.AgentState'>), writers=[ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', value='agent', skip_none=False, mapper=None), Ch

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "agent_end_2_end"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in shenzhen")]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 89, 'total_tokens': 106}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-cdba9819-2862-421e-93ff-83623a4914f8-0')]}}
----


In [3]:

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hDY06FJzZsEztfQzbVgP5l9g', 'function': {'arguments': '{"query":"current weather in Shenzhen"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 121, 'total_tokens': 142}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b468a49e-3b29-4c3d-b179-d62271ac037d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Shenzhen'}, 'id': 'call_hDY06FJzZsEztfQzbVgP5l9g'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Shenzhen\', \'region\': \'Guangdong\', \'country\': \'China\', \'lat\': 22.53, \'lon\': 114.13, \'tz_id\': \'Asia/Hong_Kong\', \'localtime_epoch\': 1717858092, \'localtime\': \'2024-06-08 22:48\'}, \'current\

## 3. Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses [Jupyter notebooks](https://jupyter.org/) and assumes the reader is as well. Jupyter notebooks are perfect interactive environments for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc), and observing these cases is a great way to better understand building with LLMs.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook. See [here](https://jupyter.org/install) for instructions on how to install.


### Installation

To install LangChain run:

In [4]:
# %pip install -U langchain-community langgraph langchain-anthropic tavily-python

For more details, see our [Installation guide](/docs/how_to/installation).

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:


## 4. Define tools


We first need to create the tools we want to use. Our main tool of choice will be [Tavily](/docs/integrations/tools/tavily_search) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717858080, 'localtime': '2024-06-08 7:48'}, 'current': {'last_updated_epoch': 1717857900, 'last_updated': '2024-06-08 07:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 11.4, 'feelslike_f': 52.6, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 10.1, 'dewpoint_f': 50.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 18.1, 'gust_kph': 29.2}}"}, {'url': 'https://forecast.weather.gov/MapClick.ph

In [6]:
search_results

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717858080, 'localtime': '2024-06-08 7:48'}, 'current': {'last_updated_epoch': 1717857900, 'last_updated': '2024-06-08 07:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 11.4, 'feelslike_f': 52.6, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 10.1, 'dewpoint_f': 50.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 18.1, 'gust_kph': 29.2}}"},
 {'url': 'https://forecast.weather.gov/MapClick

## 5. Using Language Models


Next, let's learn how to use a language model by to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fce39f96d30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fce396a73a0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')

You can call the language model by passing in a list of messages. By default, the response is a `content` string.

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [9]:
model_with_tools = model.bind_tools(tools)
model_with_tools

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fce39f96d30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fce396a73a0>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}}]})

In [10]:
model_with_tools.kwargs

{'tools': [{'type': 'function',
   'function': {'name': 'tavily_search_results_json',
    'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
    'parameters': {'type': 'object',
     'properties': {'query': {'description': 'search query to look up',
       'type': 'string'}},
     'required': ['query']}}}]}

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Shenzhen?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Shenzhen'}, 'id': 'call_FCXcsGV6MhaxMQj6cjit1TvQ'}]


In [13]:
json.loads(response.json())


{'content': '',
 'additional_kwargs': {'tool_calls': [{'id': 'call_FCXcsGV6MhaxMQj6cjit1TvQ',
    'function': {'arguments': '{"query":"current weather in Shenzhen"}',
     'name': 'tavily_search_results_json'},
    'type': 'function'}]},
 'response_metadata': {'token_usage': {'completion_tokens': 21,
   'prompt_tokens': 85,
   'total_tokens': 106},
  'model_name': 'gpt-4o',
  'system_fingerprint': 'fp_319be4768e',
  'finish_reason': 'tool_calls',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-d0f0628b-b34d-425e-8181-c2717426f6ad-0',
 'example': False,
 'tool_calls': [{'name': 'tavily_search_results_json',
   'args': {'query': 'current weather in Shenzhen'},
   'id': 'call_FCXcsGV6MhaxMQj6cjit1TvQ'}],
 'invalid_tool_calls': [],
 'usage_metadata': None}

We can see that there's now no text content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually calll it, we'll want to create our agent.

## 6. Create the agent



Now that we have defined the tools and the LLM, we can create the agent. We will be using [LangGraph](/docs/concepts/#langgraph) to construct the agent. 
Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initalize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_tool_calling_executor` will call `.bind_tools` for us under the hood.

In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)
agent_executor

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 0x7fce391a3ad0>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']), ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)]), 'agent': PregelNode(config={'tags': []}, channels={'messages': 'messages', 'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>}, triggers=['start:agent', 'tools'], mapper=functools.partial(<function _coerce_state at 0x7fce395ba700>, <class 'langgraph.prebuilt.chat_agent_executor.AgentState'>), writers=[ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', value='agent', skip_none=False, mapper=None), Ch

## 7. Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions). Note that the agent will return the **final** state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='248bb529-0cc1-4083-9fe2-374bfc227650'),
 AIMessage(content='Hello! How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-fd208fa8-4284-4854-ac06-6213211f01a7-0')]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [LangSmith trace](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

Let's now try it out on an example where it should be invoking the tool

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Guangzhou?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in Guangzhou?', id='700cf1f4-8c0a-43fd-abc8-f3d993e1109c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yqcVaPFOX2BL1cCy9FzZHELT', 'function': {'arguments': '{"query":"current weather in Guangzhou"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ac916010-fa3d-48f1-be86-dc1cb05b7103-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Guangzhou'}, 'id': 'call_yqcVaPFOX2BL1cCy9FzZHELT'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Guangzhou\', \'region\': \'Guangdong\', \'country\': \'China\', \'lat\': 23.12, \'lon\': 113.25, \'tz_id\': \'Asia/Shanghai\', \'localtime_epoch\': 1717858110,

We can check out the [LangSmith trace](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

## 8. Streaming Messages

We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Guangzhou?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_brOWkvihpimRIdNGL6iyOKOM', 'function': {'arguments': '{"query":"current weather in Guangzhou"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8de29c98-633c-4a2c-a95a-73d4c3cf3127-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Guangzhou'}, 'id': 'call_brOWkvihpimRIdNGL6iyOKOM'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Guangzhou\', \'region\': \'Guangdong\', \'country\': \'China\', \'lat\': 23.12, \'lon\': 113.25, \'tz_id\': \'Asia/Shanghai\', \'localtime_epoch\': 1717858110, \'localtime\': \'2024-06-08 22:48\'}, \'current

## 9. Streaming tokens

In addition to streaming back messages, it is also useful to be streaming back tokens.
We can do this with the `.astream_events` method.

:::{.callout-important}
This `.astream_events` method only works with Python 3.11 or higher.
:::

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/Users/wenke/.conda/envs/dl/lib/python3.9/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'current weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717858080, 'localtime': '2024-06-08 7:48'}, 'current': {'last_updated_epoch': 1717857900, 'last_updated': '2024-06-08 07:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.6, 'wind_kph': 22.0, 'wind_degree': 250, 'wind_dir': 'WSW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 96, 'cloud': 100, 'feelslike_c': 11.4, 'feelslike_f': 52.6, 'windchill_c': 10.6, 'windchill_f': 51.1, 'heatindex_c': 12.0, 'heatindex_f': 53.6, 'dewpoint_c': 1

## 10. Adding in memory


As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [19]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-8096f7cd-1001-4708-bb8f-f198fb8db3b7-0')]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned that your name is Bob. How can I help you today, Bob?', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 106, 'total_tokens': 124}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-86e8be1e-1bfa-4806-9ba0-b06577e70108-0')]}}
----


Example [LangSmith trace](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If I want to start a new conversation, all I have to do is change the `thread_id` used

In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to your personal information, including your name. If you tell me your name, I can use it in our conversation.", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 84, 'total_tokens': 113}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b1d9d702-24a6-4de5-b28c-a796326188bd-0')]}}
----


## 11. Conclusion


That's a wrap! In this quick start we covered how to create a simple agent. 
We've then shown how to stream back a response - not only the intermediate steps, but also tokens!
We've also added in memory so you can have a conversation with them.
Agents are a complex topic, and there's lot to learn! 

For more information on Agents, please check out the [LangGraph](/docs/concepts/#langgraph) documentation. This has it's own set of concepts, tutorials, and how-to guides.